In [27]:
import sys, os, importlib
sys.path.append(os.path.abspath(".."))

import src.data.nba_client as nba_client
importlib.reload(nba_client)

from src.data.nba_client import get_team_game_logs_season
df = get_team_game_logs_season("2024-25", ("Regular Season","Playoffs"))
df["TEAM_NAME"].nunique(), df["TEAM_ID"].nunique()   # should be (30, 30)
df["GAME_DATE"].min(), df["GAME_DATE"].max()
len(df)  # ~2460 for regular season; +playoffs if included


2628

In [28]:
df["TEAM_NAME"].nunique(), df["TEAM_ID"].nunique()  # expect (30, 30)
df["GAME_DATE"].min(), df["GAME_DATE"].max()


(Timestamp('2024-10-22 00:00:00'), Timestamp('2025-06-22 00:00:00'))

In [29]:
import pandas as pd

df = df.sort_values(["TEAM_NAME", "GAME_DATE"])
df["PTS_r5"] = (
    df.groupby("TEAM_NAME")["PTS"]
      .transform(lambda s: s.rolling(5, min_periods=3).mean())
)


In [ ]:
def predict_matchup(team_a: str, team_b: str, frame: pd.DataFrame):
    last = frame.groupby("TEAM_NAME").tail(1).set_index("TEAM_NAME")
    pa = float(last.loc[team_a, "PTS_r5"])
    pb = float(last.loc[team_b, "PTS_r5"])
    return {
        "team_a": team_a, "predicted_pts_a": round(pa, 1),
        "team_b": team_b, "predicted_pts_b": round(pb, 1),
        "winner": team_a if pa > pb else team_b
    }

predict_matchup("Chicago Bulls", "Cleveland Cavaliers", df)


KeyError: 'PTS_r5'

In [31]:
from nba_api.stats.endpoints import TeamGameLogs
import pandas as pd

logs = TeamGameLogs(
    season_nullable="2024-25",
    season_type_nullable="Regular Season",
    league_id_nullable="00"
)
df = logs.get_data_frames()[0]
df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])

# 1) See the raw column names
list(df.columns)

['SEASON_YEAR',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'FG3M_RANK',
 'FG3A_RANK',
 'FG3_PCT_RANK',
 'FTM_RANK',
 'FTA_RANK',
 'FT_PCT_RANK',
 'OREB_RANK',
 'DREB_RANK',
 'REB_RANK',
 'AST_RANK',
 'TOV_RANK',
 'STL_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'PTS_RANK',
 'PLUS_MINUS_RANK',
 'AVAILABLE_FLAG']

In [32]:
import sys, os, importlib
sys.path.append(os.path.abspath(".."))

from src.data.nba_client import get_team_game_logs_season
from src.features.team import basic_context, add_four_factors, attach_opponent, add_rolling, add_home_away_rolling_pts
from src.models.baselines import predict_matchup_baseline
from src.models.baselines import predict_matchup_baseline_homeaway



df = get_team_game_logs_season("2024-25", ("Regular Season","Playoffs"))
df = basic_context(df)
df = add_four_factors(df)
df = attach_opponent(df)
df = add_rolling(df, window=10)
df = add_home_away_rolling_pts(df, window=10)


predict_matchup_baseline("Boston Celtics", "Dallas Mavericks", df, col="PTS_r10")
predict_matchup_baseline_homeaway("Boston Celtics", "Dallas Mavericks", df, window=10)



ImportError: cannot import name 'add_home_away_rolling_pts' from 'src.features.team' (/Users/ssjraz/NBA-Prediction-Model/src/features/team.py)